Erin Ruby and Courtney Solano presenting 5th

# Final#
#### CSCI 4831 - Sabermetrics ####
#### Instructor: Hoenigman ####
#### Due: Wednesday, May 1 by 3pm ####


# Comments from Rhonda #
I wonder if you could consider whether the previous play increased win expectancy as a way to weight the WE on the next play. You would use (WE_Current/WE_previous) to give you a value around 1. Greater than 1 means they improved and <1 means they decreased. Use that to weight the next WE calculation. I have no idea what that would do, but it could be interesting. It might be noisy and you might want to consider WE over the previous 2 plays instead of 1, or previous 3 plays.

# iheartbaseball #

## Erin Ruby and Courtney Solano ##

A new and improved way to calculate win expectancy.  
(https://www.loom.com/share/9d10bf6e88d049a4b61be0971d7476df)
# Explanation of statistic #

Win Expectancy is the percent chance that a team will win, given a current state, right before a player bats in the game. The current win expectancy (http://www.tangotiger.net/welist.html) is calculated using historical data, looking at the percent of teams that have won, given a current state. The states consist of the inning number, whether is it is the top or bottom of the inning, if the team is tied or winning or losing by 1 point, how many outs there are, and runners on base. 

The current win expectancy is in a table, completed by Tom Tango, with some of the possible combinations of inning, top/bottom, home team score compared to the away team, outs and runners on base. This win expectancy calculation is limited because it was all done by hand and only shows states from the bottom of the 7th to the bottom of the 9th. It also only shows states when the scores are tied or have a difference of 1. Another limitation with this win expectancy calculation is that the team playing is not taken into consideration. There are teams that are better than others, which would mean that they have a higher percent chance of winning. If a really good team plays a really bad team, the better team's win expectancy should be higher, even if they find themselves in the same situation. All the scores are from the home team's perspective. 

Our win expectancy solves all of these problems, and does more! We calculate win expectancy using historical data, but we can do it for any state the user types in, and any team matchup. We used the rankings from Baseball Reference in 2018 of the teams to add more weight to their win expectancy (https://www.baseball-reference.com/leagues/MLB/2018-standings.shtml). The ranking is a simple rating system that weights all games equally, and weights all points equally, and ignores wins and losses, so the ranking is solely based on how good a team performs. It does not take into account injuries, weather conditions, or importance of the game, it solely looks at points scored and points allowed. A team with a higher ranking will have a lower index, and when normalized, a higher ranking score. We also take the previous state into consideration in the win expectancy calculation because 2 good states in a row can signify that more good states are possible in the future indicating the team is gaining momentum, therefore increasing the team's chance of winning.

Our win expectancy is calculated using historical data and adding weights to each probability. 

$$WE = \frac{P(current)}{P(previous)} + P(current) + rank$$ 

Where *P(current)* is the probability that the team will win, given the *current* state, *P(previous)* is the probability that the team will win, given the *previous* state, and *rank* is the given team's ranking normalized by dividing **0.1** by it. The *P(current)/P(previous)* is positive if it results in a number greater than **1** and negative if the value is less than **1**. We did this because if the probability of winning given a current state is less than the probability of winning given the previous state, we expect the win expectancy to go down because the team just completed a worse play. It also gets normalized by dividing by **100**. We used these normalizing constants because we found they created a measure that is very close to a probability. Our statistic can be greater than one or less than one because we are adding and subtracting constants to a probability, but it is very rare that this happens, and can be interpreted as a probability in most situations.

# Demonstration #

To run the code locally, run in your terminal-
```
cd form_ui
python get_WE.py
```
Copy and paste the URL provided in the terminal. 

In order for the code to run, you will need to have statcast data from 2016-2018 downloaded and saved as a CSV in a directory called data. 

The user enters the home and away team, if it is the top or bottom of the inning, the inning number, and the two states. The states include the home and away score, number of outs, and which bases have a runner on them. The win expectancy is calculated for the team that is up to bat, (bottom - home, top - away).

# Evaluation of statistic #

Tom Tango's tables are generic and limited. He only has values for bottom of the 7th-9th innings. He has each combination listed out, with outs and runners on base. He also only evaluates with the score as either tied or a difference of 1. This is where our statistic improved the win expectancy.  

Our win expectancy calculation is better than Tom Tango's statistic because it is more accurate. Take this example: Rockies are playing at home at the bottom of the 7th inning, game is tied, there are no outs and no runners on base. Tom Tango gives this scenario a **0.548**. This means that the home team is favored to win. Our statistic gives this scenario a **0.275**, meaning the Rockies are most likely going to lose. There was a game on June 18th, 2017 at Coors Field, where this exact scenario happened and the Rockies lost. Our statistic predicted this with much more accuracy than Tom Tango's. By taking more factors into account than just historical data, we are able to get better intuition and accuracy on if a team will win given a specific state, and the state that happened previous to that state.


# Limitations #

A limitation for our win expectancy calculation is that it only works for two consecutive states within the same inning. So, if a user wanted to get the win expectancy for the first at-bat in an inning, our code would not allow this. Also, our statistic only uses a couple of years of data, because it takes a very long time to read in. Therefore, the numbers are not quite as accurate for the unmodified win expectancy than we would like. We could have hard coded Tom Tango's tables instead, but that would take years, and we wanted to be able to use any inning and any difference in score that has happened in the past. Because we use such specific criteria to narrow down the data to the state the user inputs, there is much less data, and therefore it is not as accurate as it could be with much more data.




In [4]:
from pybaseball import statcast
from pybaseball import playerid_reverse_lookup
from pybaseball.lahman import *
from pybaseball.retrosheet import *
import pandas as pd
import numpy as np
import math
# download_lahman() 
pd.set_option('display.max_columns', 5000)

In [5]:
import matplotlib.pyplot as plt
import scipy.stats as stats
%matplotlib inline


In [19]:
def readData_Erin():
#     april2015 = pd.read_csv("../data/2015/dataApril2015.csv")
#     may2015 = pd.read_csv("../data/2015/dataMay2015.csv")
#     june2015 = pd.read_csv("../data/2015/dataJune2015.csv")
#     july2015 = pd.read_csv("../data/2015/dataJuly2015.csv")
#     aug2015 = pd.read_csv("../data/2015/dataAug2015.csv")
#     sep2015 = pd.read_csv("../data/2015/dataSept2015.csv")
#     oct2015 = pd.read_csv("../data/2015/dataOct2015.csv")
#     df2015 = pd.concat([april2015,may2015,june2015,july2015,aug2015,sep2015,oct2015])
    
    df2016 = pd.read_csv("../data/2016/data2016.csv")
    
#     april2017 = pd.read_csv("../data/2017/dataApril2017.csv")
#     may2017 = pd.read_csv("../data/2017/dataMay2017.csv")
#     june2017 = pd.read_csv("../data/2017/dataJune2017.csv")
#     july2017 = pd.read_csv("../data/2017/dataJuly2017.csv")
#     aug2017 = pd.read_csv("../data/2017/dataAug2017.csv")
#     sep2017 = pd.read_csv("../data/2017/dataSept2017.csv")
#     oct2017 = pd.read_csv("../data/2017/dataOct2017.csv")
#     df2017 = pd.concat([april2017,may2017,june2017,july2017,aug2017,sep2017,oct2017])
    
#     april2018 = pd.read_csv("../data/2018/dataApril2018.csv")
#     may2018 = pd.read_csv("../data/2018/dataMay2018.csv")
#     june2018 = pd.read_csv("../data/2018/dataJune2018.csv")
#     july2018 = pd.read_csv("../data/2018/dataJuly2018.csv")
#     aug2018 = pd.read_csv("../data/2018/dataAug2018.csv")
#     sep2018 = pd.read_csv("../data/2018/dataSept2018.csv")
#     oct2018 = pd.read_csv("../data/2018/dataOct2018.csv")
#     df2018 = pd.concat([april2018,may2018,june2018,july2018,aug2018,sep2018,oct2018])
    
#     df1518 = pd.concat([df2015, df2016, df2017, df2018])
    
    return df2016

In [7]:
def readData_Court():
    df2015 = pd.read_csv("../data/2015/data2015.csv")
    df2016 = pd.read_csv("../data/2016/data2016.csv")
    df2017 = pd.read_csv("../data/2017/data2017.csv")
    df2018 = pd.read_csv("../data/2018/data2018.csv")
    
    df1518 = pd.concat([df2015, df2016, df2017, df2018])
    
    return df1518

In [8]:
def getState(df, inning, inning_topbot, on_1b, on_2b, on_3b, outs_when_up, score):
    
    if on_1b == 1:
        df_criteria = df.loc[df['on_1b'].notnull()]
    else:
        df_criteria = df.loc[df['on_1b'].isnull()]
    if on_2b == 1:
        df_criteria = df_criteria.loc[df['on_2b'].notnull()]
    else:
        df_criteria = df_criteria.loc[df['on_2b'].isnull()]
    if on_3b == 1:
        df_criteria = df_criteria.loc[df['on_3b'].notnull()]
    else:
        df_criteria = df_criteria.loc[df['on_3b'].isnull()]
        
    df_criteria = df_criteria.loc[(df['inning']==inning) & (df['inning_topbot']==inning_topbot)
                         & ((df['home_score'])-(df['away_score'])==score)
                         & (df['outs_when_up']==outs_when_up)]
    return len(df_criteria.loc[df_criteria['W'] == True])/len(df_criteria)

In [9]:
def getRank(ranking, team):
    r = ranking.loc[ranking['Tm']==team]['Rk']
    return 0.1/r

In [10]:
def getWeight(probPrev, probCurrent, r):
    weight = (probCurrent/probPrev)
    if (weight>1):
        weight = (probCurrent/probPrev)/100
    else:
        weight = -1*(probCurrent/probPrev)/100

    return probCurrent + weight + r

In [11]:
def validStates(inning1, inning2, outs1, out2, topbot1, topbot2):
    if(inning1 < inning2): #cant go back an inning
        return False
    if ((topbot1 == topbot2) and (inning1 == inning2) ): #cant go down in outs, if in the same inning
        if(outs1 < outs2):
            return False
    if(topbot1 == 'Bot' and topbot2 =='Top'): #cant go from bot to top in the same inning
        if(inning1 >= inning2):
            return False
    else:
        return True
        
    

In [14]:
def main():
#     df1518 = readData_Erin()
#     gameData = df1518.copy().groupby(['game_date', 
#                                       'away_team', 'home_team']).max()[['away_score', 'home_score']].reset_index()
#     gameData['W'] = gameData['home_score']>gameData['away_score']
#     gameData.head()
# #     states = df1518.copy()
#     df = states.merge(gameData[['game_date','home_team','W']], how='inner',on=['game_date', 'home_team'])
#     def getState(df, inning, inning_topbot, on_1b, on_2b, on_3b, outs_when_up):
    
    ###ASK USER###
    team = 'BOS'
#     ranking = pd.read_csv('../data/rankings.csv') #https://www.baseball-reference.com/leagues/MLB/2018-standings.shtml
    probPrev = getState(df, 7, 'Top', 0, 0, 0, 0 , -1)
    probCurrent = getState(df, 7, 'Top', 0, 0, 0, 1, -1)
    r = getRank(ranking, team)
    newProb = getWeight(probPrev, probCurrent, r)
    print (newProb)

In [16]:
# main()

In [21]:
df1518 = readData_Erin()
gameData = df1518.copy().groupby(['game_date',
                                  'away_team', 'home_team']).max()[['away_score', 'home_score']].reset_index()
gameData['W'] = gameData['home_score']>gameData['away_score']
states = df1518.copy()
df = states.merge(gameData[['game_date','home_team','W']], how='inner',on=['game_date', 'home_team'])
ranking = pd.read_csv('../data/rankings.csv') #https://www.baseball-reference.com/leagues/MLB/2018-standings.shtml


In [22]:
team = 'BOS'
probPrev = getState(df, 7, 'Top', 0, 0, 0, 0 , -1)
probCurrent = getState(df, 7, 'Top', 0, 0, 0, 1, -1)
r = getRank(ranking, team)
newProb = getWeight(probPrev, probCurrent, r)
print (newProb)

0    0.23397
Name: Rk, dtype: float64


todo:
make a simple user interface to input home team, away team, which team you want to know the WE for. and 2 states (inning, outs, runners on base, difference in score) negative means home is losing. 

also we need to implement a check to see that two states can be consecutive   